In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

#读取数据
df = pd.read_csv("input/train_data.csv")  
data = np.array(df['铁路客运量_当期值(万人)'])

In [2]:
# 数据预处理
normalized_data = (data - np.mean(data)) / np.std(data)

seq_size = 3
train_x, train_y = [], []
for i in range(len(normalized_data) - seq_size - 1):
    train_x.append(np.expand_dims(normalized_data[i : i + seq_size], axis=1).tolist())
    train_y.append(normalized_data[i + 1 : i + seq_size + 1].tolist())

input_dim = 1
X = tf.placeholder(tf.float32, [None, seq_size, input_dim])
Y = tf.placeholder(tf.float32, [None, seq_size])

In [3]:
# regression
def ass_rnn(hidden_layer_size=6):
    W = tf.Variable(tf.random_normal([hidden_layer_size, 1]), name='W')
    b = tf.Variable(tf.random_normal([1]), name='b')
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer_size)
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    W_repeated = tf.tile(tf.expand_dims(W, 0), [tf.shape(X)[0], 1, 1])
    out = tf.batch_matmul(outputs, W_repeated) + b
    out = tf.squeeze(out)
    return out

In [4]:
#训练模型
def train_rnn():
    out = ass_rnn()
    loss = tf.reduce_mean(tf.square(out - Y))
    train_op = tf.train.AdamOptimizer(learning_rate=0.003).minimize(loss)

    saver = tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        #tf.get_variable_scope().reuse_variables()
        sess.run(tf.global_variables_initializer())
        for step in range(10000):
            _, loss_ = sess.run([train_op, loss], feed_dict={X: train_x, Y: train_y})
            if step % 10 == 0:
                # 用测试数据评估loss
                print("第%s步：loss:%f"%(step, loss_))
        print("保存模型: ", saver.save(sess, 'output/ass.model'))

train_rnn()

第0步：loss:1.008181
第10步：loss:0.898428
第20步：loss:0.799835
第30步：loss:0.706534
第40步：loss:0.619687
第50步：loss:0.543197
第60步：loss:0.479007
第70步：loss:0.426936
第80步：loss:0.385822
第90步：loss:0.354111
第100步：loss:0.330070
第110步：loss:0.311839
第120步：loss:0.297577
第130步：loss:0.285755
第140步：loss:0.275328
第150步：loss:0.265698
第160步：loss:0.256591
第170步：loss:0.247930
第180步：loss:0.239745
第190步：loss:0.232108
第200步：loss:0.225074
第210步：loss:0.218656
第220步：loss:0.212824
第230步：loss:0.207536
第240步：loss:0.202749
第250步：loss:0.198425
第260步：loss:0.194526
第270步：loss:0.191014
第280步：loss:0.187855
第290步：loss:0.185014
第300步：loss:0.182456
第310步：loss:0.180153
第320步：loss:0.178076
第330步：loss:0.176202
第340步：loss:0.174513
第350步：loss:0.172990
第360步：loss:0.171619
第370步：loss:0.170387
第380步：loss:0.169281
第390步：loss:0.168287
第400步：loss:0.167392
第410步：loss:0.166584
第420步：loss:0.165851
第430步：loss:0.165181
第440步：loss:0.164565
第450步：loss:0.163994
第460步：loss:0.163461
第470步：loss:0.162961
第480步：loss:0.162488
第490步：loss:0.162039
第500步：loss: